In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importing required libraries


In [ ]:
import os
import cv2
import math
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import load_model
from tensorflow import keras
import warnings
warnings.filterwarnings("ignore")

#Defining paths for inputting data

In [ ]:
# Path to root folder
root_dir = '/content/drive/MyDrive/slices/'

# Path for training data
train_dir = os.path.join(root_dir, 'train')
train_image_dir = os.path.join(train_dir, 'Images')
train_label_dir = os.path.join(train_dir, 'Labels')
train_predict_dir = os.path.join(train_dir, 'Predictions/')

# Path for validation data
validate_dir = os.path.join(root_dir, 'validate/')
validate_image_dir = os.path.join(validate_dir, 'Images')
validate_label_dir = os.path.join(validate_dir, 'Labels')
validate_predict_dir = os.path.join(validate_dir, 'Predictions/')

# Path for testing data
test_dir = os.path.join(root_dir, 'test')
test_image_dir = os.path.join(test_dir, 'Images')
test_predict_dir = os.path.join(test_dir, 'Predictions/')

#Defining some constants

In [ ]:
SEED = 909
TRAIN_BATCH_SIZE = 64
VALIDATION_BATCH_SIZE = 64
TEST_BATCH_SIZE = 64

TRAIN_SIZE = 21894
VALIDATE_SIZE = 6551
TEST_SIZE = 6531

HEIGHT = 128
WIDTH = 128
IMAGE_SIZE = (HEIGHT,WIDTH)

#Creating DataGenerators

In [ ]:
# Training generator
def training_generator(imgPath):
    data_gen_args = dict(rescale=1./255
#                      featurewise_center=True,
#                      featurewise_std_normalization=True,
#                      rotation_range=90
#                      width_shift_range=0.2,
#                      height_shift_range=0.2,
#                      zoom_range=0.3
                        )
    datagen = ImageDataGenerator(**data_gen_args)
    
    image_generator = datagen.flow_from_directory(imgPath, target_size=IMAGE_SIZE, class_mode=None, color_mode='grayscale', batch_size=1, seed=SEED, shuffle=False)
    return image_generator

# Validation generator
def validation_generator(imgPath):
    data_gen_args = dict(rescale=1./255)
    datagen = ImageDataGenerator(**data_gen_args)
    
    image_generator = datagen.flow_from_directory(imgPath, target_size=IMAGE_SIZE, class_mode=None, color_mode='grayscale', batch_size=1, seed=SEED, shuffle=False)
    return image_generator

# Testing generator
def testing_generator(imgPath):
    data_gen_args = dict(rescale=1./255)
    datagen = ImageDataGenerator(**data_gen_args)
    
    image_generator = datagen.flow_from_directory(imgPath, target_size=IMAGE_SIZE, class_mode=None, color_mode='grayscale', batch_size=1, seed=SEED, shuffle = False)
    return image_generator

In [ ]:
train_generator=training_generator(train_image_dir)
validate_generator=validation_generator(validate_image_dir)
test_generator=testing_generator(test_image_dir)

Found 21894 images belonging to 1 classes.
Found 6551 images belonging to 1 classes.
Found 6531 images belonging to 1 classes.


#Loading saved model

In [ ]:
model = load_model("/content/drive/MyDrive/slices/results/model.h5")

#Saving predictions for training, validation and testing data

In [ ]:
def save_predictions(datagen, samples, imagePath, predictPath):
  path = os.path.join(imagePath, 'Images/')
  files = os.listdir(path)
  files.sort(reverse=False)
  for i, file in zip(range(0,samples), files):
    img = cv2.imread(os.path.join(path,file))
    a, b = os.path.split(path+file)
    x, y = os.path.splitext(b)
    image = next(datagen)
    pred = model.predict(image)[0] > 0.5
    pred = np.squeeze(pred, axis=None)
    print("Saving " + str(b) + " file")
    plt.imsave(os.path.join(predictPath, str(x) + ".png"), pred, cmap='gray')

In [ ]:
save_predictions(train_generator, TRAIN_SIZE, train_image_dir, train_predict_dir) # Passing training data

In [ ]:
save_predictions(validate_generator, VALIDATE_SIZE, validate_image_dir, validate_predict_dir) # Passing valiation data

In [ ]:
save_predictions(test_generator, TEST_SIZE, test_image_dir, test_predict_dir) # Passing test data